In [1]:
import pandas as pd
import os
import datetime

In [2]:
ROOT_DIR = "C:\_Finalyca\cmots_data\mutual"
target_date = datetime.date(2022, 11, 5)
DIR_PATH = os.path.join(ROOT_DIR, target_date.strftime("%Y%m%d"))
file_prefix = target_date.strftime("%d-%m-%Y")
def get_file_name(file_name, date):
    sl = file_name.split('.')
    ext = sl.pop()
    new_filename = F"{'.'.join(sl)}_{date.strftime('%d-%m-%Y')}.{ext}"
    return new_filename


# DIR_PATH = "CMOTS_29_09_2022"
# target_date = None
# def get_file_name(file_name, date):
#     return file_name

In [3]:
def get_df(dir_path, file_name, target_date, total_columns, not_used_columns):
    f_name = get_file_name(file_name, target_date)
    total_file_name = os.path.join(DIR_PATH, f_name)
    if os.path.isfile(total_file_name):
        df = pd.read_csv(total_file_name, sep ='|', names = total_columns, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        df = df.drop(not_used_columns, axis=1)
    else:
        df = pd.DataFrame()
    return df

In [4]:
# mfadd = get_df( DIR_PATH, "mfadd.csv", target_date, 
#                ["MF_COCODE", "LNAME", "OFFICETYPE", "SRNO", "ADD1", "ADD2", "ADD3", "TEL", "FAX", "EMAIL", "WEBSITE", "FLAG"], 
#                [])
# dbdmcomp = get_df( DIR_PATH, "dbdmcomp.csv", target_date, ["Co_code", "Lname", "Co_name", "Ind_Code", "Flag"], ["Ind_Code"] )

dbdmfm = get_df( DIR_PATH, "dbdmfm.csv", target_date, ["MF_COCODE", "STARTED_ON", "AMC_CODE", "FLAG"], [])
mutfund = get_df( DIR_PATH, "mutfund.csv", target_date, 
                 ["FUND_CODE", "FUND_NAME", "FUND_TYPE", "INCORPORAT", "AMC", "ADD_1", "ADD_2", "ADD_3", "CITY", "PIN_CODE", 
                  "TEL_NO_1", "TEL_NO_2", "FAX_NO", "WEBSITE_UR", "E_MAIL"], 
                 ["INCORPORAT", "CITY", "PIN_CODE", "TEL_NO_2"])
amc = pd.merge(mutfund, dbdmfm, how="left", left_on="FUND_CODE", right_on="MF_COCODE")
amc = amc.drop(["MF_COCODE", "FLAG"], axis=1)

KeyError: 'MF_COCODE'

In [ ]:
dbdfundm = get_df(DIR_PATH, "dbdfundm.csv", target_date, ["FUNDCODE", "FUND_MGR", "FLAG"], [])
dbdprof = get_df( DIR_PATH, "dbdprof.csv", target_date, ["FUNDCODE", "QUAL", "DESIG", "EXPERIENCE", "REMARKS", "FLAG"], [])
mffundmgr = get_df( DIR_PATH, "mffundmgr.csv", target_date, ["mf_schcode", "fundcode", "fund_mgr"], [])

fund_manager= pd.merge(dbdfundm.drop("FLAG", axis=1), dbdprof.drop("FLAG", axis=1), how="outer", left_on="FUNDCODE", right_on="FUNDCODE")

In [6]:
dbdbmim = get_df( DIR_PATH, "dbdbmim.csv", target_date, ["BMCODE", "CO_CODE", "SC_CODE", "SC_NAME", "FLAG"], ["SC_CODE", "FLAG"])
dbdindic = get_df( DIR_PATH, "dbdindic.csv", target_date, ["MF_COCODE", "MF_SCHCODE", "BMCODE", "FLAG"], ["FLAG"])
bm = pd.merge(dbdindic, dbdbmim, how="left", on="BMCODE")

catmas_sebi = get_df( DIR_PATH, "catmas_sebi.csv", target_date, ["mf_schcode", "catcode", "catname", "subcatcode", "subcatname"], [])
mfriskometer = get_df( DIR_PATH, "mfriskometer.csv", target_date, ["mf_schcode", "riskometercode", "riskometervalue"], [])
mfisin_all = get_df( DIR_PATH, "mfisin_all.csv", target_date, ["mf_schcode", "ISIN"], [])
mfdirindir = get_df( DIR_PATH, "mfdirindir.csv", target_date, ["mf_schcode", "investment"], [])

schclass = get_df( DIR_PATH, "schclass.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCH_NAME", "MCLASSCODE", "MCLASS", "SCHEME", "SCHTYPCODE", "SCH_TYPE", "CLASS", "CLASS1", 
                   "PLAN", "DIVFRQ", "SECTOR", "VCLASSCODE", "VCLASS", "flag"], 
                  ["MCLASSCODE", "MCLASS", "CLASS1", "SECTOR", "flag"]
                 )
amfimaster = get_df(DIR_PATH, "amfimaster.csv", target_date, 
                    ["MF_COCODE", "MF_SCHCODE", "SRNO", "SCH_NAME", "AMFICODE", "FLAG"], 
                    ["SRNO", "FLAG"])
mfcat = get_df( DIR_PATH, "mfcat.csv", target_date, 
               ["MF_COCODE", "GRPCODE", "MF_SCHCODE", "GRPNAME", "MF_SCHNAME", "PLANNAME", "MPLANNAME", "OPTIONNAME", "MOPTIONNAME"],
               [])
d = pd.merge(catmas_sebi, mfriskometer, how="left", on=["mf_schcode"])
d = pd.merge(d, mfisin_all, how="left", on=["mf_schcode"])
d = pd.merge(d, mfdirindir, how="left", on=["mf_schcode"])

d1 = pd.merge(schclass, amfimaster, how="left", on=["MF_COCODE", "MF_SCHCODE", "SCH_NAME"])
mfcat = mfcat.drop(["MF_SCHNAME"], axis=1)

d1 = pd.merge(d1, mfcat, how="left", on=["MF_COCODE", "MF_SCHCODE"])

plans = pd.merge(d1, d, how="left", left_on="MF_SCHCODE",right_on="mf_schcode")
plans = plans.drop(["mf_schcode"], axis=1)
plans = pd.merge(plans, bm, how="left", on=["MF_COCODE", "MF_SCHCODE"])

In [ ]:
dbdprice = get_df( DIR_PATH, "dbdprice.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "NAVDATE", "NAVRS", "ADJNAVRS", "NAVREPRICE", "NAVSAPRICE", "NAVLSPRICE", "RC", "SC", "FLAG"],
                  ["NAVLSPRICE", "RC", "SC"])

dbdprice_n = get_df( DIR_PATH, "dbdprice_n.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "NAVDATE", "NAVRS", "ADJNAVRS", "NAVREPRICE", "NAVSAPRICE", "NAVLSPRICE", "RC", "SC", "FLAG"],
                  ["NAVLSPRICE", "RC", "SC"])

nav = dbdprice
nav_n = dbdprice_n

In [ ]:
dbdpfoli = get_df( DIR_PATH, "dbdpfoli.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "TYPECODE", "STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING", "PERIOD", 
                   "PEROFCODE", "TOTNAV", "MKTVALUE", "REPSECTOR", "INV_NO", "CO_CODE", "CO_NAME", "NO_SHARES", "PERC_HOLD", "INTPER", "EDATE",
                   "GROUP", "COMCODE", "COMNAME", "TCODE", "TNAME", "TOPPF", "FLAG"], 
                  ["STYPECODE", "DEBTYPCODE", "PEROFCODE", 
                   "EDATE", "GROUP", "TOPPF"])

dbdnptm = get_df( DIR_PATH, "dbdnptm.csv", target_date, 
                 ["COMCODE", "COMNAME", "TYPECODE", "TYPE", "SUBCODE", "SUBTYPNAME", "DEBTYPCODE", "DEBTYPE", 
                  "LEV1", "LEV2", "LEV3", "OLD1", "OLD2", "OLD3"],
                 ["LEV1", "LEV2", "LEV3", "OLD1", "OLD2", "OLD3"])

dbdderiv = get_df( DIR_PATH, "dbdderiv.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "TYPECODE", "STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING", "PERIOD", 
                   "PEROFCODE", "TOTNAV", "MKTVALUE", "REPSECTOR", "INV_NO", "CO_CODE", "CO_NAME", "NO_SHARES", "PERC_HOLD", "INTPER", 
                   "COMCODE", "COMNAME", "TCODE", "TNAME", "FLAG"], 
                  ["STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING",
                   "PEROFCODE", "INTPER", "TCODE", "TNAME", "FLAG"])

# Derivatives are not used as of now
holdings = pd.merge(dbdpfoli, dbdnptm, how="left", on=["TYPECODE", "COMCODE"])

In [ ]:
# cardv = get_df( DIR_PATH, "cardv.csv", target_date, 
#                [ "DATE", "SCHCLCODE", "SCHCLASS", "1DAYMAX", "1DAYMIN", "1DAYAVG", "WEEKMAX", "WEEKMIN", "WEEKAVG", "1MMAX", "1MMIN", "1MAVG", 
#                 "3MMAX", "3MMIN", "3MAVG", "6MMAX", "6MIN", "6MAVG", "1YRMAX", "1YRMIN", "1YRAVG", "3YRMAX", "3YRMIN", "3YRAVG", 
#                 "5YRMAX", "5YRMIN", "5YRAVG", "INCMAX", "INCMIN", "INCAVG"],
#                [ "1DAYMAX", "1DAYMIN", "WEEKMAX", "WEEKMIN", "1MMAX", "1MMIN", "3MMAX", "3MMIN", "6MMAX", "6MIN", "1YRMAX", "1YRMIN", 
#                 "3YRMAX", "3YRMIN", "5YRMAX", "5YRMIN", "INCMAX", "INCMIN"
#                ]
#               )

# catmas = get_df(DIR_PATH, "catmas.csv", target_date, ["VCLASSCODE", "VCLASS", "AssetType1", "AssetType2"], [])

# dbddiv = get_df( DIR_PATH, "dbddiv.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "DIVTYPCODE", "DIVTYPE", "NAVREFDATE", "BONUSRATE1", "BONUSRATE2", "DIVDATE", "DIVPER", 
#     "CDIVPER", "NCDIVPER", "EXNAV", "DIVAMT", "DIVACPER", "DIVACPDATE", "DIVACP", "DIVACFDATE", "DIVPERPU", "RECORDDATE", "REMARK", 
#     "INTDATE", "FV", "FLAG"],
#                ["EXNAV", "DIVAMT", "DIVACPDATE", "DIVACFDATE", "INTDATE"]
#                )

# dbddivtm = get_df( DIR_PATH, "dbddivtm.csv", target_date, [
#     "DIVTYPCODE", "DIVTYPE", "FLAG"],
#                   []
#                  )

# dbdaum = get_df( DIR_PATH, "dbdaum.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "AMCDATE", "AMCSIZE", "FLAG"
# ], [])

# dbdexpr = get_df( DIR_PATH, "dbdexpr.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "ERDATE", "EXPRATIO", "EXPAMT", "UNITCAP", "NOOFUNITS", "FLAG"
# ], [])

# latnav = get_df( DIR_PATH, "latnav.csv", target_date, 
#                 ["MF_SCHCODE","NAVDATE","NAVRS","NAVREPRICE","NAVSAPRICE","NAVLSPRICE","CLDATE","SCH_NAME","LNAME","CO_CODE"], 
#                 ["NAVLSPRICE","CLDATE","LNAME"])

# latnav_n = get_df( DIR_PATH, "latnav_n.csv", target_date, 
#                 ["MF_SCHCODE","NAVDATE","NAVRS","NAVREPRICE","NAVSAPRICE","NAVLSPRICE","CLDATE","SCH_NAME","LNAME","CO_CODE"], 
#                 ["NAVLSPRICE","CLDATE","LNAME"])

In [ ]:
dbdload2 = get_df( DIR_PATH, "dbdload2.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "LOADDATE", "LOADTYPE", "LOAD_NO", "FRDATE", "TODATE", "FRAMT", "UPTOAMT", "MINPERIOD", "MAXPERIOD", 
    "MIN", "MAX", "ENTRYLOAD", "EXITLOAD", "REMARKS", "FLAG"],
                 [])

dbdmfsch = get_df( DIR_PATH, "dbdmfsch.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SC_CODE", "INIT_PRICE", "OFFERPRICE", "SCHEME", "SCHTYPCODE", "LAUNC_DATE", "CLDATE", "INCDATE", 
                   "SIZE", "SIZE_DATE", "DIVPAYINT", "REDEM_DATE", "FUNDCODE", "FORM_AVAIL", "LIST_UNLST", "SECTCODE", "AMT_RAISED", "OBJECTIVE", 
                   "MININVT", "INC_INVEST", "PURREDEM", "NAVCALC", "TAXBCODE", "SPFEATURE", "SCHCLCODE", "NOTES", "FLAG"], 
                  ["SC_CODE", "FORM_AVAIL", "LIST_UNLST", "SECTCODE", "AMT_RAISED", "TAXBCODE", "SPFEATURE", "SCHCLCODE"]
                 )

returns = get_df( DIR_PATH, "return.csv", target_date, 
                 ["MF_COCODE", "MF_SCHCODE", "SCH_NAME", "DATE", "TODATE", "TONAV", "TONAVP", "PNAVFROM", "PNAV", "PNAVP", "PRET", 
                  "1WEEKFRM", "1WEEKNAV", "1WEEKNAVP", "1WEEK", "1MTHFRM", "1MTHNAV", "1MTHNAVP", "1MONTH", "3MTHFRM", "3MTHNAV", "3MTHNAVP", "3MONTH", 
                  "6MNTFROM", "6MNTNAV", "6MNTNAVP", "6MONTH", "1YRFROM", "1YRNAV", "1YRNAVP", "1YEAR", "3YRFROM", "3YRNAV", "3YRNAVP", "3YEAR", 
                  "5YRFROM", "5YRNAV", "5YRNAVP", "5YEAR", "INCFROM", "INCNAV", "INCEPTION", 
                  "FUNDSIZE", "SCHCLCODE", "CLASSNAME", "TYPECODE", "AMC_CODE", "AMCNAME", "NAVDATE", "NAVRS", "LAUNCHDT"], 
                 ["TONAVP", "PNAVFROM", "PNAV", "PNAVP", "PRET", "1WEEKNAVP", "1MTHNAVP", "3MTHNAVP", "6MNTNAVP", "1YRNAVP", "3YRNAVP",
                  "5YRNAVP", "NAVDATE", "NAVRS"]
                )

cardsebi = get_df( DIR_PATH, "cardsebi.csv", target_date, 
                  ["DATE", "subcat_code", "subcate_name", "1DAYMAX", "1DAYMIN", "1DAYAVG", "WEEKMAX", "WEEKMIN", "WEEKAVG", "1MMAX", "1MMIN", "1MAVG",
                   "3MMAX", "3MMIN", "3MAVG", "6MMAX", "6MIN", "6MAVG", "1YRMAX", "1YRMIN", "1YRAVG", "3YRMAX", "3YRMIN", "3YRAVG", 
                   "5YRMAX", "5YRMIN", "5YRAVG", "INCMAX", "INCMIN", "INCAVG"], 
                  ["1DAYMAX", "1DAYMIN", "WEEKMAX", "WEEKMIN", "1MMAX", "1MMIN", "3MMAX", "3MMIN", "6MMAX", "6MIN", "1YRMAX", "1YRMIN",
                   "3YRMAX", "3YRMIN", "5YRMAX", "5YRMIN", "INCMAX", "INCMIN"])

dbdavgm = get_df( DIR_PATH, "dbdavgm.csv", target_date, [
     "MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "AVG_MAT1", "AVG_MAT2", "YTM", "AVG_PF1", "AVG_PF2", "FLAG"
], [])

dbdmprof = get_df( DIR_PATH, "dbdmprof.csv", target_date, [
     "MF_COCODE", "MF_SCHCODE", "INVDATE", "TYPECODE", "MAT_NO", "P1", "P2", "P3", "P4", "PERC_HOLD", "MKTVALUE", "FLAG"
], [])

dbdptm = get_df( DIR_PATH, "dbdptm.csv", target_date, [
    "TYPECODE", "TYPE", "FLAG"
], [])

dbdtor = get_df( DIR_PATH, "dbdtor.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "BMCODE", "FLAG"
], [])

dbdtor = get_df( DIR_PATH, "dbdtor.csv", target_date, [
    "mf_cocode", "mf_schcode", "torDate", "torRatio", "torTimes", "flag"
], [])

DBDPLIV1 = get_df( DIR_PATH, "DBDPLIV1.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "LOAD", "REMARK", "COTPURCH", "COTREDEM", "FLAG"
], [])

DBDPLIV2 = get_df( DIR_PATH, "DBDPLIV2.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "AVAIL", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "ANY", "OTHERS", "FLAG"
], [])

DBDPLIV3 = get_df( DIR_PATH, "DBDPLIV3.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "FREQ", "SLNO", "MINAMT", "MINUNIT", "MULTAMT", "MULTUNIT", "INSTLMNT", "DURATION", "FLAG"
], [])

amcsiz = get_df( DIR_PATH, "amcsiz.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "INVDATE", "TOTNAV", "FLAG"
], [])

DBDYTDER = get_df( DIR_PATH, "DBDYTDER.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "YTDERDATE", "YTDER", "ER", "WAER", "FLAG"
], [])

In [ ]:
plans

In [ ]:
cardsebi

In [7]:
schclass

,MF_COCODE,MF_SCHCODE,SCH_NAME,SCHEME,SCHTYPCODE,SCH_TYPE,CLASS,PLAN,DIVFRQ,VCLASSCODE,VCLASS
0,5946,1,Aditya Birla SL Equity Hybrid '95 Fund (G),OPEN,1,Growth,Balanced,Equity Oriented,ANNUAL,13,Hybrid - Equity Oriented
1,5946,2,Aditya Birla SL Equity Hybrid '95 Fund (IDCW),OPEN,2,Dividend,Balanced,Equity Oriented,INTERIM,13,Hybrid - Equity Oriented
2,5946,7,Aditya Birla SL Tax Relief '96 (IDCW),OPEN,2,Dividend,Equity,Tax Planning,NaN,3,Equity - Tax Planning
3,5946,8,Aditya Birla SL Low Duration Fund (G),OPEN,1,Growth,Ultra Short Term Funds,Regular,NaN,33,Ultra Short Term Funds
4,5946,9,Aditya Birla SL Flexi Cap Fund (G),OPEN,1,Growth,Equity,Diversified,NaN,1,Equity - Diversified
...,...,...,...,...,...,...,...,...,...,...,...
8178,21271,43416,IDFC CRISIL IBX 90:10 SDL Plus Gilt-Apr 2032 I...,OPEN,2,Dividend,Income Funds,NaN,NaN,17,Income Funds
8179,18585,43417,DSP FMP - Series 268 - 1281Days (G),CLOSE,1,Growth,Fixed Maturity Plans,NaN,NaN,24,Fixed Maturity Plans
8180,18585,43418,DSP FMP - Series 268 - 1281Days (IDCW),CLOSE,2,Dividend,Fixed Maturity Plans,NaN,NaN,24,Fixed Maturity Plans
8181,18585,43419,DSP FMP - Series 268 - 1281Days - Direct (G),CLOSE,1,Growth,Fixed Maturity Plans,NaN,NaN,24,Fixed Maturity Plans


In [8]:
mfcat

,MF_COCODE,GRPCODE,MF_SCHCODE,GRPNAME,PLANNAME,MPLANNAME,OPTIONNAME,MOPTIONNAME
0,5946,ABSL1,1,Aditya Birla SL Equity Hybrid '95 Fund,Regular,Regular,Growth,Growth
1,5946,ABSL1,2,Aditya Birla SL Equity Hybrid '95 Fund,Regular,Regular,Dividend,Dividend
2,5946,ABSL7324,7,Aditya Birla SL Tax Relief '96,Regular,Regular,Dividend,Dividend
3,5946,ABSL8,8,Aditya Birla SL Low Duration Fund,Regular,Regular,Growth,Growth
4,5946,ABSL9,9,Aditya Birla SL Flexi Cap Fund,Regular,Regular,Growth,Growth
...,...,...,...,...,...,...,...,...
8188,21271,IDFC43413,43416,IDFC CRISIL IBX 90:10 SDL Plus Gilt-Apr 2032 I...,Regular,Regular,Dividend,Dividend
8189,18585,DSP43417,43417,DSP FMP - Series 268 - 1281Days,Regular,Regular,Growth,Growth
8190,18585,DSP43417,43418,DSP FMP - Series 268 - 1281Days,Regular,Regular,Dividend,Dividend
8191,18585,DSP43417,43419,DSP FMP - Series 268 - 1281Days,Regular,Regular,Growth,Growth
